In [29]:
import numpy as np
import pandas as pd
import pickle
from gensim.models import word2vec
import MeCab
import re


tagger = MeCab.Tagger(" -r /etc/mecabrc -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")

def tokenize_ja(text, lower):
    node = tagger.parseToNode(str(text))
    while node:
        # 分かち書きで取得する品詞を指定
        if lower and node.feature.split(',')[0] in ["名詞", "形容詞"]:
            yield node.surface.lower()
        node = node.next


def tokenize(content, token_min_len, token_max_len, lower):
    return [
        str(token) for token in tokenize_ja(content, lower)
        if token_min_len <= len(token) <= token_max_len and not token.startswith('_')
    ]


# 学習データの読み込み
df = pd.read_csv('./data/200_ramen_review.csv')
df2 = pd.read_csv('./data/200_ramen_store.csv')
df = pd.merge(df, df2, on="ID")
df_ramen = df.groupby(['name', 'score', 'review_count'])['review'].apply(
    list).apply(' '.join).reset_index().sort_values('score', ascending=False)



In [4]:
df_ramen

,name,score,review_count,review
28,らぁ麺や 嶋,3.99,249,特製醤油らぁ麺 1180円あの”ラーメンの鬼”佐野氏のDNAを継ぐお店が西新宿に！…という情...
0,Homemade Ramen 麦苗,3.98,758,特上 塩つけ 959月10日、今日の昼食は多摩川を越えて大森まで遠...
112,手打式超多加水麺 ののくら,3.96,532,食べログのラーメン部門で東京No.1になった「ののくら」にやってまいりました～！（2020年...
183,麺尊 RAGE,3.94,784,金曜夜19:15着。外待ち4名に接続。外待ち席にて、注文を聞かれました。そこで、ご飯もの終了...
108,宍道湖しじみ中華蕎麦 琥珀,3.94,346,前からものすごくいってみたかったお店！ただし、平日しかやっていないのと雑色という立地の不便さ...
...,...,...,...,...
159,馳走麺 狸穴,3.72,743,2021年3月28日（日）本日はニボ美のリクエストでこちらの店を初訪問。12時02分に到着す...
77,中華そば 青葉 中野本店,3.72,727,中野でラーメン屋と言ったら、往年のラーメンファンがまず思い浮かぶのはココですよね中野行く度に...
55,ラーメン二郎 目黒店,3.72,575,かなり久しぶりのラーメン二郎目黒店開店30分前で外待ち15人30分前ですぐシャッターが開き、...
129,煮干中華そば鈴蘭,3.72,555,～限定煮干そば・味噌 800円～去年９月より体絞って２２ｋｇ痩せた僕です。久々のレビューやけ...


In [30]:
# コーパス作成
def preprocessing_text(text):
    # 改行、半角スペース、全角スペースを削除
    text = re.sub('\r', '' , text)
    text = re.sub('\n', '' , text)
    text = re.sub('　', '' , text)
    text = re.sub(' ', '' , text)
    text = text.lower()
    # 数字文字の一律「0」化
    text = re.sub('\d+,?\d*', '0', text) 
    return text
                  
wakati_ramen_text = []
for i in df_ramen['review']:
    txt = preprocessing_text(i)
    txt = tokenize(txt, 2, 10000, True)
    wakati_ramen_text.append(txt)

np.savetxt("./data/200_ramen_corpus.txt", wakati_ramen_text,
           fmt='%s', delimiter=',', encoding='UTF-8')
np.savetxt("./data/200_ramen_corpus.txt", wakati_ramen_text,
           fmt='%s', delimiter=',', encoding='UTF-8')


/opt/conda/lib/python3.9/site-packages/numpy/lib/npyio.py:1378: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.asarray(X)


In [31]:
# モデル作成

word2vec_ramen_model = word2vec.Word2Vec(wakati_ramen_text,sg=1, vector_size=100, window=5,min_count=5,workers=3)

word2vec_ramen_model = word2vec.Word2Vec(wakati_ramen_text, sg = 1, vector_size=100, window = 5, min_count = 5, workers = 3)

#sg（0: CBOW, 1: skip-gram）,size（ベクトルの次元数）,window（学習に使う前後の単語数）,min_count（n回未満登場する単語を破棄）,iter（トレーニング反復回数）

# モデルのセーブ
word2vec_ramen_model.save("./model/word2vec_ramen_model_v2.model")

In [32]:
# モデルのロード
#model(ver1)
word2vec_ramen_model =word2vec.Word2Vec.load("./model/word2vec_ramen_model.model")
word2vec_ramen_model.wv.most_similar("二郎")

[('日々', 0.9057350754737854),
 ('ファン', 0.8976778984069824),
 ('インスパイア', 0.889609158039093),
 ('熱烈', 0.8873732686042786),
 ('長浜ラーメン', 0.8848977088928223),
 ('経験', 0.884236216545105),
 ('お気に入り', 0.8769047260284424),
 ('中野', 0.8761041760444641),
 ('取材拒否', 0.8754292130470276),
 ('思い出', 0.8753508925437927)]

In [33]:
# モデルのロード
#model(ver2)
word2vec_ramen_model =word2vec.Word2Vec.load("./model/word2vec_ramen_model_v2.model")
word2vec_ramen_model.wv.most_similar("二郎")

[('野猿', 0.8096699118614197),
 ('二郎系', 0.8089856505393982),
 ('ラーメン二郎', 0.7650819420814514),
 ('三田', 0.7554795742034912),
 ('乳化', 0.7452131509780884),
 ('博多ラーメン', 0.7447754144668579),
 ('野猿街道', 0.7325755953788757),
 ('家系', 0.7277393341064453),
 ('インスパイア', 0.7213176488876343),
 ('直系', 0.7164747714996338)]

In [53]:
word2vec_ramen_model.wv.most_similar("山岸")

[('アルバイト', 0.9619793891906738),
 ('代目', 0.9582072496414185),
 ('小川', 0.9562837481498718),
 ('長谷川', 0.9525606036186218),
 ('タレント', 0.9504513740539551),
 ('本人', 0.9504405856132507),
 ('郡山', 0.9489341378211975),
 ('定食屋', 0.9486766457557678),
 ('運営', 0.9464150667190552),
 ('漢字', 0.9450749754905701)]

In [49]:
word2vec_ramen_model.wv.most_similar(positive=[u"ラーメン",u"北海道"])

[('直送', 0.8479162454605103),
 ('開発', 0.8356922268867493),
 ('確立', 0.8338803052902222),
 ('order', 0.8319934010505676),
 ('久留米', 0.830234706401825),
 ('トンコツ', 0.8286890387535095),
 ('なぁ～', 0.8258898258209229),
 ('チャッチャ', 0.8250440955162048),
 ('七福', 0.8245644569396973),
 ('湯塩', 0.8235633969306946)]